# Manage Imports

In [1]:
import numpy as np
import gym
import random

# Initialize Environment

In [2]:
env = gym.make("FrozenLake-v0")

def has_won(state):
    return state == 15

# Create Q-Table

Create an $M \times N$ matrix, with 
- $M$ = size of action space
- $N$ = size of state space / observation space

In [3]:
action_space_size = env.action_space.n
observation_space_size = env.observation_space.n

qtable = np.zeros((observation_space_size, action_space_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# Custom Rewards: Table to Map Custom Rewards to State and Feature-Toggle

In [4]:
rewards = [0, 1, 1, 1, 1, -10, 1, -10, 1, 1, 1, -10, -10, 1, 1, 100]
custom_rewards = False

# Setup Hyper Parameters

In [5]:
total_training_episodes = 15000
total_play_episodes = 1000
learning_rate = 0.8
max_steps = 99

#discounting rate
gamma = 0.95

#exploration rate
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01

#exponential decay rate for exploration probability
decay_rate = 0.0005

# Output Setup

In [6]:
render = False
verbosity = 0 # 0=light, 1=medium, 2=heavy, 3=desperate housewifes

# Play Around With the Environment...

In [7]:
# env.reset()
# 
# for step in range(200):
#     observation, reward, done, info = env.step(env.action_space.sample())
#     
#     print("================")
#     print("Step {}".format(step))
#     print(info['prob'])
#     print("================")
#     
#     if (render):
#         env.render()
#     
#     if (done):
#         break

# Playground for numpy etc...

In [8]:
#a = [3, 5, 2]
#print(np.argmax(a))

#b = [[5, 2, 1], [2, 9, 1], [1, 0, -5]]
#print(np.argmax(b[0]))

#text coloring from https://stackoverflow.com/questions/287871/how-to-print-colored-text-in-terminal-in-python
#class bcolors:
#    HEADER = '\033[95m'
#    OKBLUE = '\033[94m'
#    OKGREEN = '\033[92m'
#    WARNING = '\033[93m'
#    FAIL = '\033[91m'
#    ENDC = '\033[0m'
#    BOLD = '\033[1m'
#    UNDERLINE = '\033[4m'

# print("\033[95mHEADER\033[0m")
# print("\033[94mBLUE\033[0m")
# print("\033[92mGREEN\033[0m")
# print("\033[91mFAIL\033[0m")
# print("\033[1mBOLD\033[0m")
# print("\033[1m\033[91mBOLD AND RED\033[0m normal")
# print("\033[1m\033[92mBOLD AND GREEN\033[0m normal")

# The Q-Learning Algorithm

In [9]:
total_training_reward = 0
epsilon = max_epsilon

for episode in range(total_training_episodes):
    if (verbosity > 1):
        print("===================================")
        print("Starting Episode {}".format(episode))
        
    state = env.reset()
    reward = 0
    done = False
    info = {'prob': 0}

    total_reward = 0
    step = 0
    
    while (not done and step < max_steps):        
        # step 1: choose action (explore or exploit)
        explore = random.random() < epsilon
        epsilon = np.clip((1 - decay_rate) * epsilon, min_epsilon, max_epsilon)

        # step 2: take action
        action = env.action_space.sample() if explore else np.argmax(qtable[state])
        new_state, reward, done, info = env.step(action)
        
        if (custom_rewards):
            reward = rewards[new_state]
            
        total_reward += reward
        
        if (verbosity > 2):
            print("Step: {}, Epsilon: {}, Observation: {}, Reward: {}".format(new_state, epsilon, new_state, reward))
        
        if (render):
            env.render()
            
        # step 3: update q-table
        # Q'(s, a) := Q(s, a) + lr * (R(s, a) + gamma * max(Q(s', a')) - Q(s, a))
        qtable[state][action] = qtable[state][action] + learning_rate * (reward + gamma * max(qtable[new_state]) - qtable[state, action])
            
        if (verbosity > 2):
            print(qtable)
        
        state = new_state
        step += 1
    
    total_training_reward += total_reward
    
    if (verbosity > 2 or (verbosity > 0 and has_won(state))):
        print("{}Episode {} survived for {} steps. Final Epsilon: {}\nTotal Reward (Training): {}, Average Reward (Training): {}{}\n".format("\033[1m\033[92m" if has_won(state) else "", episode, step - 1, epsilon, total_training_reward, total_training_reward / (episode + 1), ", SOLVED!!!\033[0m" if has_won(state) else ""))
    
    if (verbosity > 1):
        print("Q-Table:")
        print(qtable)
        print("Board on GameOver:")
        env.render()

# Final Q-Table

In [10]:
print(qtable)

[[4.21339014e-01 4.12605009e-02 5.29406389e-02 3.09323529e-02]
 [5.20265601e-03 1.14305290e-02 5.82031182e-03 4.01140954e-02]
 [1.45132263e-02 8.94251398e-02 1.22288154e-02 1.45725312e-02]
 [8.98790564e-03 4.59902709e-03 2.01552104e-03 3.53173053e-02]
 [4.22535830e-01 3.94484120e-02 1.92211064e-02 5.15059766e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.47744852e-01 2.07522684e-06 6.31923949e-07 4.07437278e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.16635143e-03 1.22251953e-02 2.68830564e-01 5.54247024e-01]
 [3.06263488e-02 6.80049339e-01 3.31908410e-02 3.35083554e-02]
 [7.51894126e-01 6.65173022e-03 6.82860239e-03 7.86630648e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.53703743e-01 9.26305482e-02 7.84565279e-01 6.74792966e-02]
 [5.76576216e-01 9.68879128e-01 3.10010568e-01 2.78093240e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000000

# Let the Agent Play the Game with our Q-Table now

In [11]:
successful_episodes = 0
consecutive_successes = 0
consecutive_successes_record = 0

for episode in range(total_play_episodes):
    state = env.reset()
    done = False
    step = 0
    
    while (not done):
        action = np.argmax(qtable[state])
        state, reward, done, info = env.step(action)
        step += 1
    
    if (verbosity > 0):
        print("\n{}Episode finished after {} steps. {}".format("\033[1m\033[92m" if has_won(state) else "", step, "SOLVED!!!\033[0m" if has_won(state) else ""))
        env.render()
    
    if (has_won(state)):
        successful_episodes += 1
        consecutive_successes += 1
        
        if (consecutive_successes > consecutive_successes_record):
            consecutive_successes_record = consecutive_successes
    else:
        consecutive_successes = 0

print("Successful Episodes: {}, Success Rate: {}, Most Consecutive Successes: {}".format(successful_episodes, successful_episodes / total_play_episodes, consecutive_successes_record))

Successful Episodes: 743, Success Rate: 0.743, Most Consecutive Successes: 17


# Successfull Q-Tables:

In [12]:
#[[61.04317248 59.78547346 59.93280907 59.92662166]
# [61.74434791 57.0925934  11.12133377 58.32503135]
# [58.84817333 59.85014631 48.1443941  58.89104811]
# [-0.57958262 66.51807248 52.91629263 63.71962457]
# [64.75927529 62.30205679 11.40749456 60.16229307]
# [ 0.          0.          0.          0.        ]
# [48.65363695  1.70487442 13.97564412  9.4206155 ]
# [ 0.          0.          0.          0.        ]
# [-0.90176315 60.70798534 67.19631989 75.76824235]
# [68.41653963 79.05868273 63.55264237 66.35457724]
# [82.67354435 88.05008973 -0.25917098 76.05612972]
# [ 0.          0.          0.          0.        ]
# [ 0.          0.          0.          0.        ]
# [84.75296864 14.40572824 92.506149   82.24510467]
# [90.84339089 93.64134073 99.72749531 97.59303161]
# [ 0.          0.          0.          0.        ]]

#[[0.54803756 0.58266842 0.67390149 0.57267101]
# [0.11684947 0.61331875 0.5890873  0.53338789]
# [0.24328052 0.54619072 0.63081553 0.57269565]
# [0.59131255 0.48893312 0.60379805 0.61094363]
# [0.74103326 0.63220326 0.64233009 0.57346231]
# [0.         0.         0.         0.        ]
# [0.00116312 0.02502368 0.16183577 0.11191187]
# [0.         0.         0.         0.        ]
# [0.57022986 0.02808945 0.16653554 0.61127876]
# [0.02871648 0.77690207 0.88235692 0.02904237]
# [0.79354517 0.75911488 0.87537473 0.76689976]
# [0.         0.         0.         0.        ]
# [0.         0.         0.         0.        ]
# [0.03061699 0.13995663 0.94052307 0.91944422]
# [0.88678153 0.88818483 0.82643528 0.83592829]
# [0.         0.         0.         0.        ]]

#[[6.28064964e-02 5.95541698e-02 6.25139931e-02 5.12391642e-02]
# [5.51930085e-03 5.10730668e-03 1.28591665e-02 5.96682993e-02]
# [3.50481321e-02 4.14289449e-03 1.37061701e-01 3.85170044e-02]
# [1.81377747e-03 2.75213180e-04 3.24269003e-02 3.79059265e-02]
# [1.39920680e-01 9.75866867e-04 3.47784571e-02 5.86369574e-03]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [3.37520346e-05 6.98960517e-06 5.15266076e-01 2.24836496e-05]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.16273082e-02 6.17196501e-02 1.11017589e-02 1.45181751e-01]
# [9.05981541e-02 3.38622452e-01 3.64544629e-03 3.69297725e-03]
# [5.49003744e-01 5.44286244e-04 4.03659104e-03 5.25707756e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.16180179e-01 9.43171853e-03 5.60514788e-01 1.05673672e-01]
# [1.45671731e-01 8.08115002e-01 8.24878698e-02 1.20159521e-01]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

#[[1.92994141e-01 7.54256576e-03 7.61538345e-03 1.02436121e-02]
# [5.29774122e-03 1.91508018e-03 2.01055628e-03 3.65341387e-02]
# [5.46759832e-03 5.40919853e-03 1.87444071e-03 8.84179094e-03]
# [1.67128102e-03 9.49145280e-04 5.64309421e-03 8.38467348e-03]
# [2.75068653e-01 1.24633349e-02 5.62608040e-03 3.70980775e-03]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [2.47627043e-02 6.64929411e-06 9.59666059e-06 9.21042832e-12]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [2.72446376e-03 3.00339622e-04 1.49915243e-03 3.90032117e-01]
# [1.58386684e-03 2.05663476e-01 1.31617870e-03 1.39550996e-02]
# [5.41301894e-01 2.23003606e-04 8.47214151e-04 7.34463476e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [2.43114529e-03 6.83970811e-03 3.06352245e-01 3.64940459e-02]
# [6.70653353e-02 9.21068151e-01 1.92145358e-01 1.09943165e-01]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

#[[3.85688610e-01 9.13026482e-02 8.03817500e-02 9.25942327e-02]
# [3.28683589e-03 1.16003216e-03 1.13083596e-02 8.12515790e-02]
# [1.43746799e-02 6.26830287e-03 1.50602447e-02 3.44379881e-02]
# [7.47430739e-04 4.70554404e-03 3.20537103e-03 2.44443992e-02]
# [4.57057062e-01 1.01302678e-02 2.38232019e-02 3.42845278e-02]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [3.03990277e-02 1.08096115e-06 9.01683596e-06 2.09569372e-08]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [4.64517130e-02 6.32204935e-03 5.16380127e-02 5.46633979e-01]
# [5.97567551e-03 3.29850318e-01 1.00086859e-01 3.74988328e-02]
# [4.95180528e-01 4.01733608e-04 2.42079624e-03 6.83844748e-03]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.06857941e-01 1.16388649e-01 4.90802117e-01 1.97575232e-01]
# [1.50382487e-01 9.10636738e-01 2.17253093e-01 2.09039007e-01]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

# With Custom Rewards: 
# rewards = [0, 1, 1, 1, 1, -10, 1, -10, 1, 1, 1, -10, -10, 1, 1, 100]
#[[40.96935218 20.9709767  20.89576736 19.01336981]
# [-8.45354677 -6.06096808 -5.06962222 20.34569058]
# [ 2.98202555 16.2483915  -0.47405021 20.74631372]
# [20.23022449 14.2598559  -8.98164154 20.75892057]
# [57.3127133  -4.67690139  0.51344434 -9.90974741]
# [ 0.          0.          0.          0.        ]
# [-9.99778581 -9.99588575 -3.96312241 -9.99957294]
# [ 0.          0.          0.          0.        ]
# [-8.82631708  2.7403365  -3.76437217 74.82691169]
# [-8.00439942 86.85094708 -4.42226162 -5.52909999]
# [85.61440427 -5.92489893 -9.87966738 -9.75306497]
# [ 0.          0.          0.          0.        ]
# [ 0.          0.          0.          0.        ]
# [57.04938107  4.22991713 79.53213683 55.93472361]
# [27.43454017 96.93250297 28.44238757  4.83250021]
# [ 0.          0.          0.          0.        ]]